A Sample Python code for extracting data from spareroom(https://www.spareroom.co.uk/) and storing data in a mysql database. 
Spareroom does not have an API, so I parse the website with Bueatifulsoupe package of python and
clean it in a simple way to only include my preferences. It asks the user the number of rooms,
and the location they want to look for, and store the data based on the last update. 

An example file of London data is uploaded, one can select data based on their preference from the data base. 

BY **Fatimah Ahmadi** 

In [124]:
#packages 
import mysql.connector, requests, re
from bs4 import BeautifulSoup

In [125]:
# create the database change your user and pass accordingly
cnx = mysql.connector.connect(user="root", password="@Ahmadi1368", database="rooms")
cursor = cnx.cursor()
storage_of_id = []

In [126]:
#url of website, cases are updated based on last update
url = "https://www.spareroom.co.uk/flatshare/"
sorting = "?sort_by=last_updated"

def url_func(location):
    return url+location.lower()

In [127]:
#extract ids of webpage for each case
def ides(number, location):
    main_page = url_func(location)+"/page"+str(number)+sorting
    page = requests.get(main_page)
    soup = BeautifulSoup(page.text, "html.parser")
    id_list = soup.find_all("li", attrs={"class":"listing-result"})
    ids = []
    for item in  id_list:
        ids.append(item.attrs["data-listing-id"])
    return ids

In [128]:
def count(item1, list_items):
    for item in list_items:
        if item1 in item.text:
            return item
  
    return None

In [129]:
#extract details or room and data from a page specified with an id. 
def extract_data(id_num):
    main_page = "https://www.spareroom.co.uk/flatshare/flatshare_detail.pl?featured=1&flatshare_id={}&search_id=&search_results=%2Fflatshare%2Foxford%2Fpage2%3Fsort_by%3Dlast_updated".format(id_num)
    page = requests.get(main_page)
    soup = BeautifulSoup(page.text, "html.parser")
    current_keys = ['# housemates', 'Total # rooms', 'Ages', 'Smoker?', 'Any pets?', 'Language', 'Nationality', 'Occupation','Interests','Gender']
    future_keys = ['Couples OK?', 'Smoking OK?', 'Pets OK?', 'Occupation','Housing Benefit', 'Min age', 'Max age', 'Gender']

    #Location
    loc = soup.find("section", attrs={"class":"feature feature--details"})
    if loc != None:
        loc2 = re.sub("\s+", " ", loc.text).split(" ")
        location = loc2[2]
        postal_code =  loc2[3]
    else:
        location = "Note Mentioned"
        postal_code =  "Note Mentioned"


    #Price
    price1= soup.find(attrs={"class":"room-list__price"})
    if price1 != None:
        price = "".join(c for c in price1.text if c.isdigit())
    else:
        price2= soup.find(attrs={"class":"feature feature--price-whole-property"})
        price22=price2.find("h3")
        price = "".join(c for c in price22.text if c.isdigit())
        
    #availability
    avl1=soup.find(attrs={"class":"feature feature--availability"}).text
    avl2= re.sub("\s+", " ", avl1).strip()
    avl3=re.sub("Availability", "", avl2).strip()
    Availability = avl3

    #Deposit and Bills
    dep1 = soup.find(attrs={"class":"feature feature--extra-cost"})
    dep_values = dep1.find_all(attrs={"class":"feature-list__value"})
    dep_keys = dep1.find_all(attrs={"class":"feature-list__key"})
    dep_bill = {}

    if len(dep_keys) >= 3:
        dep_keys.remove(dep_keys[1])
        dep_values.remove(dep_values[1])

    if count("Deposit", dep_keys ) == None:
        dep_bill["Deposit"] = "Not Mentioned"
    else:
        dep_bill["Deposit"] = dep_values[dep_keys.index(count("Deposit", dep_keys ))].text


    if count("Bill", dep_keys) == None:
        dep_bill["Bill"] = "Not Mentioned"
    else:
        dep_bill["Bill"] = dep_values[dep_keys.index(count("Bill", dep_keys))].text

    
    #current_household
    house = soup.find(attrs={"class":"feature feature--current-household"})
    current_mates = {}
    house_keys2 = []
    if house != None:
        house_values = house.find_all(attrs={"class":"feature-list__value"})
        house_keys = house.find_all(attrs={"class":"feature-list__key"})
        for num in range(0, len(house_keys)):
            key = re.sub("\s+", " ", house_keys[num].text.strip())
            house_keys2.append(key)
            value = re.sub("\s+", " ", house_values[num].text.strip())
            current_mates[key] = value
    else:
        for item in current_keys:
            current_mates[item] = " "


    for name in current_keys:
        if name not in house_keys2:
            current_mates[name] = " "


    #future_housemate
    mate = soup.find(attrs={"class":"feature feature--household-preferences"})
    future_mates={}
    mate_keys2=[]
    if mate != None:
        mate_values = mate.find_all(attrs={"class":"feature-list__value"})
        mate_keys = mate.find_all(attrs={"class":"feature-list__key"})

 
        for num in range(0, len(mate_keys)):
            key = re.sub("\s+", " ", mate_keys[num].text.strip())
            mate_keys2.append(key)
            value = re.sub("\s+", " ", mate_values[num].text.strip())
            future_mates[key] = value
        
    else:
        for item in future_keys:
            future_mates[item] = " "
            
    for name in future_keys:
        if name not in mate_keys2:
            future_mates[name] = " "

    #store in a single tuples
    return (id_num, location, postal_code, price, Availability, 
            dep_bill["Deposit"], dep_bill["Bill"], current_mates['# housemates'], 
            current_mates['Total # rooms'], current_mates['Ages'], current_mates['Smoker?'],  
            current_mates['Any pets?'], current_mates['Language'], current_mates['Nationality'], 
            current_mates['Occupation'], current_mates['Gender'], future_mates['Couples OK?'], 
            future_mates['Smoking OK?'], future_mates['Pets OK?'], future_mates['Occupation'], 
            future_mates['Housing Benefit'], future_mates['Min age'], future_mates['Max age'], 
            future_mates['Gender'] )

In [121]:
#create the table 
def table_maker(location, cursor, cnx):
    cursor.execute("Drop table IF EXISTS {}".format(location))
    cnx.commit()
    cursor.execute("Create Table {} (id varchar(500), Location varchar(500), Postcode varchar(500), Rent varchar(500), Availability varchar(200), Deposite varchar(500), Bills varchar(500), Housemates varchar(500), Total_Room varchar(500), Current_Age varchar(500), Current_Smoker varchar(500), Current_Pets varchar(500), Language varchar(500), Nationality varchar(500), Current_Occupation varchar(500), Current_Gender varchar(500), Future_Couples varchar(500), Future_Smoker varchar(500), Future_Pets varchar(500), Future_Occupation varchar(500), Housing_Benefits varchar(500), Future_Age_Min varchar(500), Future_Age_Max varchar(500), Future_Gender varchar(500))".format(location))
    cnx.commit();
    
#writing data into a mysql database
def writing(id_num, location, cursor, cnx):
    command = "INSERT INTO {} Values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(location)
    values = extract_data(id_num)
    cursor.execute(command, values); cnx.commit()

#filling the table
def filling_table(row, location, cursor, cnx):
    table_maker(location, cursor, cnx)
    for i in range(0, row, 10):
        list_ids = ides(i, location)
        for id_num in list_ids:
            writing(id_num, location, cursor, cnx)
    print("END of Programme")

In [122]:
row = int(input("How many cases you want to store? "))
location = input("City? ")
filling_table(row, location, cursor, cnx)

How many cases you want to store? 100
City? Oxford
END of Programme
